In [3]:
import os
import configparser
from langchain_groq import ChatGroq
from langchain_cohere import ChatCohere
from langchain_core.messages import HumanMessage, SystemMessage

config = configparser.ConfigParser()
config.read('../config.ini')
groq = config['groq']
cohere = config['cohere']

os.environ['GROQ_API_KEY'] = groq.get('GROQ_API_KEY')
os.environ['COHERE_API_KEY'] = cohere.get('COHERE_API_KEY')

In [4]:
import assignment
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate #import the Class for creating a prompt

parser = StrOutputParser()
actor = assignment.actor_picker() 
location = assignment.location_picker()
theme = assignment.theme_picker()

system_template = "generates a story centered around the chosen Actor, Location, and Theme.Please provide a {responseType} response." 

input = f"Create a story where {actor} is the main character in {location}, and the story follows a {theme} theme."
# Create prompt template instance.
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_template),
        ("user", input)
    ]
)

# prompt Template also implements runnable and can be easily chained.
model = ChatCohere(model="command-r-plus")
chain = prompt_template | model | parser

chain.invoke({"responseType":"brief"})


"Priyanka Chopra stars as Mira in this dramatic tale set in the heart of Ahmedabad. \n\nMira is a young and ambitious woman with a passion for art and a dream to leave her mark on the world. Growing up in the vibrant and cultural city of Ahmedabad, she always felt a deep connection to its rich history and traditions. \n\nAs Mira pursues her artistic dreams, she finds herself entangled in a complex web of family secrets and long-buried truths. Her once-idyllic childhood takes on a darker shade as she uncovers a hidden past that threatens to shatter her world. \n\nThe story unfolds as Mira, driven by her relentless spirit, delves into the mysteries of her family's history. She navigates a landscape of emotional revelations and long-held silences, all while trying to forge her path and make her mark on the city she calls home. \n\nAs the layers of secrets peel away, Mira discovers a strength she never knew she possessed. Her artistic passion becomes a means of self-discovery and a way to 

In [5]:
model = ChatGroq(model="llama3-8b-8192")
chain = prompt_template | model | parser

chain.invoke({"responseType":"brief"})

"Title: The Colors of Change\n\nPriyanka Chopra, a renowned actress, stepped off the train at Ahmedabad's Sabarmati railway station, the vibrant city's bustle enveloping her like a warm hug. She had arrived in search of inspiration for her next role, and the city's rich cultural heritage and tumultuous history beckoned her.\n\nAs she checked into her hotel, Priyanka was struck by the city's kaleidoscope of colors – the bright hues of the traditional Gujarati clothing, the saffron and crimson of the temples, and the dusty pink of the ancient architecture. She felt an inexplicable connection to this place, as if the stories she was about to uncover would somehow become a part of her own narrative.\n\nPriyanka began her journey by visiting the magnificent Akshardham temple, its intricate carvings and stunning architecture a testament to the city's rich cultural heritage. As she wandered through the complex, she stumbled upon a group of young girls, their faces painted with bright colors a

In [6]:
# import the chat history classes
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
import asyncio # library for writing code that interacts with DB, network calls etc. 

#Create a store in memory
store = InMemoryChatMessageHistory()


# Lets define a function that gets messages from store
async def getMessage():
    await asyncio.sleep(2) # this will mimic a read from DB
    print("Messages retrieved from DB")
    return await store.aget_messages()

# Add the initial messages to the store
store.add_message(HumanMessage(input))

# Retrieve the stored messages 
messages = await getMessage()

# Model invocation to generate a story based on the prompt
response = model.invoke(messages)  # Call the model with the messages
print("Generated Story:", response.content)  # Display the generated story

# Add the model's response to the store
store.add_message(SystemMessage(response.content))

# allow the user to customize the story

user_input = "surprising twist at the end"
user_input1 = f"change the given story according to user input {user_input}"

# Add the user's customization request to the store
store.add_message(HumanMessage(user_input1))

# Retrieve updated messages after user input
messages = await getMessage()

# Display all stored messages
print("Conversation History:", messages)

# Call the model again with the updated conversation including the user's input
response = model.invoke(messages)
print("Updated Story:", response.content)

# Add the updated model response to the store
store.add_message(SystemMessage(response.content))

Messages retrieved from DB
Generated Story: Title: The Ahmedabad Affair

Priyanka Chopra, a renowned Bollywood actress, found herself in the midst of a dramatic love triangle in Ahmedabad, the vibrant city of Gujarat. Her visit to the city was meant to be a quick publicity stint for her upcoming film, but little did she know that her life was about to take a dramatic turn.

As she stepped off the plane, Priyanka was greeted by the warm Gujarati sunshine and the sound of traditional folk music filling the air. She had been looking forward to this trip for weeks, eager to explore the city's rich cultural heritage and get a break from the hustle and bustle of Mumbai.

Her first stop was the iconic Ahmedabad Museum, where she was scheduled to launch a new exhibit on the city's history. As she walked through the exhibits, she was struck by the beauty and grandeur of the ancient structures on display. It was then that she met him - Rohan, a charming and handsome local historian who was guidi